# EWX API

In [1]:
import pandas as pd
import numpy as np
import json
import os
import time
import math
import datetime
import requests
import pprint
from pandas.io.json import json_normalize
pp = pprint.PrettyPrinter(indent = 4)

## Requests Header

Header requires a bearer token (expires every hour), namespace & response type.

In [4]:
bearer = 'ya29.ImCxB4yn9xoWQFE--kDOSc5FLQoL5fHcMUKt_nBUl9phOcxBJUHxIsu7FZXPcTN-IahDhG5IAdnVpGFULP3tnIf8cgJl4zxSk7Shy-8bfjhyeZ1q96gca_bL0-EqTZ7_A2E'
auth = ''.join(['Bearer ', bearer])
header = {'Authorization':auth, 'X-NAMESPACE':'na.engie.com', 'accept':'application/json'}

## User Get Request

In [7]:
url = "https://api.cloud.energyworx.com/_ah/api/ewx/v1/user/get"
user = requests.get(url, headers = header).json()

output = {'user_email':user['email'], 'namespace':user['namespace'], 'refresh': user['refreshToken']}
print('found user info:')
pp.pprint(output)
print('')

refresh = output['refresh']
namespaces = user['namespaces']
for n in namespaces:
    print('found namespace: {}.'.format(n['name']))
    
namespace = 'na.engie.com'

found user info:
{   'namespace': 'na.engie.com',
    'refresh': '1//04oj6cnPIUoWyCgYIARAAGAQSNwF-L9Ir4d-OFpuqHnaYEbdpIPXTr5h4YJ0e94B3kIcDWE0DLNK5sXLqHhxU8qXHXc2jpVC65FE',
    'user_email': 'steven.hurwitt@external.engie.com'}

found namespace: live.engie-na.com.
found namespace: na.engie-test.com.
found namespace: na.engie.com.


## Datasource Get Request

Function to parse properties & classifiers given datasource

In [13]:
def get_datasource(account):
    
    url = "https://api.cloud.energyworx.com/_ah/api/ewx/v1/datasource/get"
    ds_url = "/".join([url, account])

    datasource = requests.get(ds_url, headers = header).json()

    tags = datasource['tags']
    properties = []

    for t in tags:
        props = t['properties']
        for p in props:
            properties.append((p['key'], p['value']))

    properties = dict(properties)
    props_df = pd.DataFrame.from_dict(properties, orient = 'index', columns = ['properties'])

    channels = datasource['channels']
    classifiers = []

    for c in channels:
        classifiers.append((c['id'], c['classifier']))
    
    classifiers = dict(classifiers)
    class_df = pd.DataFrame.from_dict(classifiers, orient = 'index', columns = ['classifiers'])
    
    return(props_df, class_df, datasource)

In [17]:
properties, classifiers, ds = get_datasource("NEPOOL_MECO_1286952002")

In [18]:
classifiers.head()

,classifiers
DELIVERY_SCALAR,DELIVERY_SCALAR
DELIVERY_SCALAR_HEARTBEAT,DELIVERY_SCALAR_HEARTBEAT
TOTAL_DEMAND,TOTAL_DEMAND
TOTAL_DEMAND_HEARTBEAT,TOTAL_DEMAND_HEARTBEAT
DELIVERY_IDR,DELIVERY_IDR


In [19]:
ds

{'description': '',
 'tags': [{'validFrom': '1900-01-01T00:00:00.000000',
   'description': 'icap',
   'removed': False,
   'version': '2018-10-09T15:06:55.000000',
   'tag': 'icap_utility_2019_06_01',
   'createdBy': 'System',
   'tagLinks': [''],
   'properties': [{'deleted': False,
     'value': 'False',
     'readOnly': False,
     'value_type': 'String',
     'key': 'calculated',
     'type': 'String'},
    {'deleted': False,
     'value': 'True',
     'readOnly': False,
     'value_type': 'String',
     'key': 'default',
     'type': 'String'},
    {'deleted': False,
     'value': 'False',
     'readOnly': False,
     'value_type': 'String',
     'key': 'estimated',
     'type': 'String'},
    {'deleted': False,
     'value': 'S',
     'readOnly': False,
     'value_type': 'String',
     'key': 'periodstatus',
     'type': 'String'},
    {'deleted': False,
     'value': 'True',
     'readOnly': False,
     'value_type': 'String',
     'key': 'utilityprovided',
     'type': 'Strin

## Query Get Request

In [ ]:
url = "/ewx/v1/query/execute"
eql = requests.get(url, headers = header)

In [ ]:
query = """SELECT *
FROM flow_metadata
WHERE STRUCT (datasource_id, flow_timestamp) IN
  (SELECT AS STRUCT datasource_id, MAX(flow_timestamp) AS flow_timestamp 
  FROM flow_metadata
  GROUP BY datasource_id)

AND datasource_id IN (
  SELECT datasource_id
  FROM tags, UNNEST(properties) as props
  WHERE tag = 'metadata'
  AND props.key = 'market'
  AND (props.value = 'NEPOOL' OR props.value = 'NYISO')
)
ORDER BY flow_timestamp DESC"""